<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<center><font size=10>Generative AI for Business Applications</center></font>
<center><font size=6>Model Context Protocol</center></font>

# Problem Statement

## Business Context

In today's competitive marketplace, organizations across industries face mounting pressure to optimize their supply chain operations while maintaining lean inventory levels. Traditional inventory management systems often rely on manual processes, reactive decision-making, and fragmented data sources, leading to significant operational inefficiencies. Modern businesses require real-time visibility into stock levels, automated reorder processes, and intelligent demand forecasting to remain competitive.

The integration of AI-powered agents into inventory management represents a transformative opportunity to enhance operational efficiency, reduce carrying costs, and improve customer satisfaction. With supply chain disruptions becoming increasingly common and customer expectations for product availability rising, organizations need intelligent systems that can proactively manage inventory, predict demand patterns, and optimize procurement strategies.

## Problem Definition

**1. Manual Inventory Tracking and Human Error**:

* Stock Level Monitoring: Manual counting and record-keeping lead to inventory discrepancies, with studies showing 25-30% variance in manual inventory counts
* Data Entry Errors: Human error in recording transactions, quantities, and prices results in inaccurate inventory valuation and poor decision-making
* Time-Intensive Processes: Staff spend excessive time on routine inventory tasks instead of strategic activities

**2. Reactive Inventory Management**

* Stockouts and Lost Sales: Lack of proactive monitoring leads to unexpected stockouts, resulting in lost revenue and customer dissatisfaction
* Overstock Situations: Poor demand forecasting causes excess inventory, tying up working capital and increasing storage costs
* Emergency Procurement: Reactive ordering often results in higher procurement costs due to expedited shipping and supplier premiums

**3. Fragmented Information Systems**

* Data Silos: Inventory data scattered across multiple systems (ERP, WMS, POS) prevents comprehensive visibility
* Lack of Real-Time Insights: Delays in data synchronization lead to decisions based on outdated information
* Poor Integration: Disconnected systems require manual data reconciliation and increase error probability

**4. Inefficient Communication and Coordination**

* Supplier Management: Manual communication with suppliers about reorder needs, delivery schedules, and quality issues
* Internal Coordination: Poor information flow between purchasing, warehouse, and sales teams
* Alert Fatigue: Generic alerts without intelligent prioritization overwhelm staff with non-critical notifications

**5. Compliance and Cost Control Challenges**

* Regulatory Compliance: Difficulty tracking expiration dates, batch numbers, and maintaining audit trails for regulated industries
* Cost Optimization: Inability to identify optimal reorder quantities, negotiate better terms, or track total cost of ownership
* Performance Metrics: Limited visibility into key performance indicators like inventory turnover, carrying costs, and supplier performance

## Objective

To revolutionize inventory management through the implementation of an **AI-powered agent system using Model Context Protocol (MCP)** that transforms reactive inventory processes into proactive, intelligent operations, thereby maximizing operational efficiency, minimizing costs, and enhancing customer satisfaction.

# Importing and installing the necessary libraries

In [1]:
!pip install langchain==0.3.27 \
            langchain-openai==0.3.0 \
            langchain-mcp-adapters==0.1.14 \
            langgraph==1.0.1

**Note:**
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in this notebook.

In [1]:
from mcp.server.fastmcp import FastMCP
from typing import List, Dict, Any, Optional
from datetime import datetime
import json
import os
import asyncio
import subprocess
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langchain import hub

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Creating Utility Functions

The code defines an inventory for a store (mock database), detailing information such as product name, category, current quantity, minimum threshold, price, and supplier for each item, along with transaction history detailing purchases and sales.

In [3]:
inventory = {
    "LAPTOP001": {
        "name": "Dell Latitude 7420",
        "category": "Electronics",
        "quantity": 15,
        "min_threshold": 5,
        "price": 1200.00,
        "supplier": "Dell Technologies",
        "last_updated": "2025-01-15",
        "transactions": [
            {"date": "2025-01-10", "type": "purchase", "quantity": 20, "unit_cost": 1150.00},
            {"date": "2025-01-12", "type": "sale", "quantity": 5, "unit_price": 1200.00}
        ]
    },
    "CHAIR042": {
        "name": "Ergonomic Office Chair",
        "category": "Furniture",
        "quantity": 8,
        "min_threshold": 3,
        "price": 350.00,
        "supplier": "Office Depot",
        "last_updated": "2025-01-14",
        "transactions": [
            {"date": "2025-01-05", "type": "purchase", "quantity": 12, "unit_cost": 320.00},
            {"date": "2025-01-08", "type": "sale", "quantity": 4, "unit_price": 350.00}
        ]
    },
    "PAPER001": {
        "name": "A4 Copy Paper (500 sheets)",
        "category": "Office Supplies",
        "quantity": 2,
        "min_threshold": 10,
        "price": 8.99,
        "supplier": "Staples",
        "last_updated": "2025-01-13",
        "transactions": [
            {"date": "2025-01-01", "type": "purchase", "quantity": 50, "unit_cost": 7.50},
            {"date": "2025-01-10", "type": "sale", "quantity": 48, "unit_price": 8.99}
        ]
    },
    "MONITOR055": {
        "name": "Samsung 27\" 4K Monitor",
        "category": "Electronics",
        "quantity": 12,
        "min_threshold": 4,
        "price": 450.00,
        "supplier": "Samsung",
        "last_updated": "2025-01-16",
        "transactions": [
            {"date": "2025-01-15", "type": "purchase", "quantity": 15, "unit_cost": 400.00},
            {"date": "2025-01-16", "type": "sale", "quantity": 3, "unit_price": 450.00}
        ]
    }
}

The `check_stock` function generates a detailed stock report for a specific item in the inventory by its item code, indicating the current stock level, threshold status, price, total value, supplier, and the last updated date, alongside a status icon and message to represent stock conditions as "CRITICAL," "LOW," or "GOOD."

In [4]:
def check_stock(item_code: str) -> str:
    """Check current stock level and details for a specific item"""
    item = inventory.get(item_code)
    if not item:
        return f"Item code '{item_code}' not found in inventory."

    status_icon = "🔴" if item['quantity'] <= item['min_threshold'] else "🟡" if item['quantity'] <= item['min_threshold'] * 2 else "🟢"
    stock_status = "CRITICAL" if item['quantity'] <= item['min_threshold'] else "LOW" if item['quantity'] <= item['min_threshold'] * 2 else "GOOD"

    total_value = item['quantity'] * item['price']

    return f"""
{status_icon} Stock Report: {item['name']} ({item_code})
─────────────────────────────────────────────
Current Stock: {item['quantity']} units
Minimum Threshold: {item['min_threshold']} units
Status: {stock_status}
Unit Price: ${item['price']:.2f}
Total Value: ${total_value:.2f}
Supplier: {item['supplier']}
Last Updated: {item['last_updated']}
"""

The `add_stock` function facilitates the addition of stock (purchase or restock) for a specified item by its item code, updating the inventory with the new quantity, last updated date, and recording the transaction, while also providing a summary of the operation including the units added and their cost.

In [5]:
def add_stock(item_code: str, quantity: int, unit_cost: Optional[float] = None) -> str:
    """Add stock to inventory (purchase/restock operation)"""
    if item_code not in inventory:
        return f"Item code '{item_code}' not found. Use create_item() first."

    if quantity <= 0:
        return "Quantity must be greater than 0."

    item = inventory[item_code]
    old_quantity = item['quantity']
    item['quantity'] += quantity
    item['last_updated'] = datetime.now().strftime("%Y-%m-%d")

    # Record transaction
    cost = unit_cost or item['price'] * 0.85  # Default to 85% of selling price
    transaction = {
        "date": datetime.now().strftime("%Y-%m-%d"),
        "type": "purchase",
        "quantity": quantity,
        "unit_cost": cost
    }
    item['transactions'].append(transaction)

    return f"""
Stock Added Successfully!
─────────────────────────
Item: {item['name']} ({item_code})
Added: {quantity} units
Stock Level: {old_quantity} → {item['quantity']} units
Unit Cost: ${cost:.2f}
Total Cost: ${cost * quantity:.2f}
"""

The `remove_stock` function manages the removal of stock from the inventory for a particular item code, ensuring sufficient stock is available, updating the inventory, recording the transaction as a sale, and providing a detailed summary of the removal, including the potential warning if the new stock level falls below the minimum threshold.

In [6]:
def remove_stock(item_code: str, quantity: int, unit_price: Optional[float] = None) -> str:
    """Remove stock from inventory (sale/usage operation)"""
    if item_code not in inventory:
        return f"Item code '{item_code}' not found."

    if quantity <= 0:
        return "Quantity must be greater than 0."

    item = inventory[item_code]
    if item['quantity'] < quantity:
        return f"Insufficient stock! Available: {item['quantity']}, Requested: {quantity}"

    old_quantity = item['quantity']
    item['quantity'] -= quantity
    item['last_updated'] = datetime.now().strftime("%Y-%m-%d")

    # Record transaction
    price = unit_price or item['price']
    transaction = {
        "date": datetime.now().strftime("%Y-%m-%d"),
        "type": "sale",
        "quantity": quantity,
        "unit_price": price
    }
    item['transactions'].append(transaction)

    # Check if stock is below threshold
    warning = ""
    if item['quantity'] <= item['min_threshold']:
        warning = f"\n WARNING: Stock below minimum threshold ({item['min_threshold']} units)!"

    return f"""
Stock Removed Successfully!
─────────────────────────
Item: {item['name']} ({item_code})
Removed: {quantity} units
Stock Level: {old_quantity} → {item['quantity']} units
Unit Price: ${price:.2f}
Total Revenue: ${price * quantity:.2f}{warning}
"""

The `get_low_stock_alerts` function generates a report listing inventory items with stock levels that are critically low (below or equal to the minimum threshold) or low (up to twice the minimum threshold), providing alerts for each category to facilitate timely reordering and maintain adequate stock levels.

In [7]:
def get_low_stock_alerts() -> str:
    """Get list of items that are low on stock or below minimum threshold"""
    low_stock_items = []
    critical_items = []

    for code, item in inventory.items():
        if item['quantity'] <= item['min_threshold']:
            critical_items.append((code, item))
        elif item['quantity'] <= item['min_threshold'] * 2:
            low_stock_items.append((code, item))

    if not low_stock_items and not critical_items:
        return "All items are well-stocked!"

    alert_msg = "Stock Alerts\n" + "="*50 + "\n"

    if critical_items:
        alert_msg += "\nCRITICAL - Reorder Immediately:\n"
        for code, item in critical_items:
            alert_msg += f"  • {item['name']} ({code}): {item['quantity']} units (Min: {item['min_threshold']})\n"

    if low_stock_items:
        alert_msg += "\n🟡 LOW STOCK - Consider Reordering:\n"
        for code, item in low_stock_items:
            alert_msg += f"  • {item['name']} ({code}): {item['quantity']} units (Min: {item['min_threshold']})\n"

    return alert_msg

The `get_inventory_summary` function provides a comprehensive summary of the inventory, optionally filtered by a specific category, detailing the total number of items, units, and value of items in the inventory, along with a category-wise breakdown if applicable.

In [8]:
def get_inventory_summary(category: Optional[str] = None) -> str:
    """Get inventory summary, optionally filtered by category"""
    filtered_items = inventory.items()

    if category:
        filtered_items = [(code, item) for code, item in inventory.items()
                         if item['category'].lower() == category.lower()]
        if not filtered_items:
            return f"❌ No items found in category '{category}'"

    total_value = 0
    total_items = 0
    categories = {}

    summary = f"Inventory Summary"
    if category:
        summary += f" - {category.title()}"
    summary += "\n" + "="*60 + "\n"

    for code, item in filtered_items:
        item_value = item['quantity'] * item['price']
        total_value += item_value
        total_items += item['quantity']

        cat = item['category']
        if cat not in categories:
            categories[cat] = {'items': 0, 'value': 0, 'quantity': 0}
        categories[cat]['items'] += 1
        categories[cat]['value'] += item_value
        categories[cat]['quantity'] += item['quantity']

    # Category breakdown
    for cat, stats in categories.items():
        summary += f"\n{cat}:\n"
        summary += f"   Items: {stats['items']} types\n"
        summary += f"   Total Units: {stats['quantity']}\n"
        summary += f"   Total Value: ${stats['value']:,.2f}\n"

    summary += f"\n Grand Total: ${total_value:,.2f}"
    summary += f"\n Total Units: {total_items:,}"
    summary += f"\n Unique Items: {len(filtered_items)}"

    return summary

The `get_transaction_history` function retrieves and displays the most recent transactions for a specified item by its item code, showing up to a given limit of transactions, sorted by date, with details such as date, type, quantity, unit price, and total transaction value.

In [9]:
def get_transaction_history(item_code: str, limit: int = 10) -> str:
    """Get recent transaction history for an item"""
    if item_code not in inventory:
        return f" Item code '{item_code}' not found."

    item = inventory[item_code]
    transactions = item['transactions']

    if not transactions:
        return f"No transaction history found for {item['name']} ({item_code})."

    # Sort by date (most recent first)
    sorted_transactions = sorted(transactions, key=lambda x: x['date'], reverse=True)
    limited_transactions = sorted_transactions[:limit]

    history = f"Transaction History: {item['name']} ({item_code})\n"
    history += "="*60 + "\n"

    for i, trans in enumerate(limited_transactions, 1):
        if trans['type'] == 'purchase':
            amount = f"${trans['unit_cost']:.2f} each"
            total = f"${trans['unit_cost'] * trans['quantity']:.2f}"
        else:
            amount = f"${trans['unit_price']:.2f} each"
            total = f"${trans['unit_price'] * trans['quantity']:.2f}"

        history += f"{i:2}. {trans['date']} | {trans['type'].title()} | "
        history += f"{trans['quantity']} units @ {amount} = {total}\n"

    if len(transactions) > limit:
        history += f"\n... and {len(transactions) - limit} more transactions"

    return history

The `create_item` function adds a new item to the inventory with specified details including item code, name, category, initial quantity, price, minimum threshold, and supplier, while validating the input values and optionally recording an initial stock transaction if there is a positive quantity.

In [10]:
def create_item(item_code: str, name: str, category: str, initial_quantity: int,
                price: float, min_threshold: int, supplier: str) -> str:
    """Create a new item in the inventory"""
    if item_code in inventory:
        return f"Item code '{item_code}' already exists."

    if initial_quantity < 0 or price <= 0 or min_threshold < 0:
        return "Invalid values. Quantity and threshold must be non-negative, price must be positive."

    inventory[item_code] = {
        "name": name,
        "category": category,
        "quantity": initial_quantity,
        "min_threshold": min_threshold,
        "price": price,
        "supplier": supplier,
        "last_updated": datetime.now().strftime("%Y-%m-%d"),
        "transactions": []
    }

    # Add initial stock transaction if quantity > 0
    if initial_quantity > 0:
        inventory[item_code]['transactions'].append({
            "date": datetime.now().strftime("%Y-%m-%d"),
            "type": "purchase",
            "quantity": initial_quantity,
            "unit_cost": price * 0.8  # Assume 80% cost ratio
        })

    return f"""
New Item Created Successfully!
─────────────────────────────────
Item Code: {item_code}
Name: {name}
Category: {category}
Initial Stock: {initial_quantity} units
Price: ${price:.2f}
Min Threshold: {min_threshold} units
Supplier: {supplier}
"""

The `get_inventory_dashboard` function provides an overall summary of the inventory, showcasing key statistics including the total number of item types, the total value of the inventory, and the number of items with low stock. It also categorizes items by category and outlines actions required for low-stock items, along with directions for checking transaction history.

In [11]:
def get_inventory_dashboard() -> str:
    """Get a comprehensive inventory dashboard"""
    total_items = len(inventory)
    total_value = sum(item['quantity'] * item['price'] for item in inventory.values())
    low_stock_count = sum(1 for item in inventory.values() if item['quantity'] <= item['min_threshold'])

    categories = {}
    for item in inventory.values():
        cat = item['category']
        if cat not in categories:
            categories[cat] = 0
        categories[cat] += 1

    dashboard = f"""
INVENTORY DASHBOARD
{'='*50}

Quick Stats:
  └─ Total Items: {total_items} types
  └─ Total Value: ${total_value:,.2f}
  └─ Low Stock Alerts: {low_stock_count} items

Categories:
"""

    for cat, count in categories.items():
        dashboard += f"  └─ {cat}: {count} items\n"

    dashboard += f"""
Action Required:
  └─ Items needing reorder: {low_stock_count}
  └─ Use get_low_stock_alerts() for details

Recent Activity:
  └─ Use get_transaction_history() for item details
"""

    return dashboard

The `get_supplier_contacts` function compiles and returns a directory of contact information for suppliers associated with items in the inventory, using mock data for supplier contacts and listing them in a formatted manner while noting any missing contact details.

In [12]:
def get_supplier_contacts() -> str:
    """Get supplier contact information"""
    suppliers = set(item['supplier'] for item in inventory.values())

    # Mock contact info (in real app, this would be from a database)
    contact_info = {
        "Dell Technologies": "1-800-DELL-TECH | orders@dell.com",
        "Office Depot": "1-800-OFFICE | business@officedepot.com",
        "Staples": "1-800-STAPLES | supplies@staples.com",
        "Samsung": "1-800-SAMSUNG | b2b@samsung.com"
    }

    contacts = "Supplier Contact Directory\n" + "="*40 + "\n"

    for supplier in sorted(suppliers):
        contacts += f"\n{supplier}\n"
        contacts += f"   {contact_info.get(supplier, ' Contact info not available')}\n"

    return contacts


# Creating Server-Side Logic

Stitching all the utility functions mentioned above in the server

In [13]:
# -- SERVER SIDE --

%%writefile server.py
from mcp.server.fastmcp import FastMCP
from typing import List, Dict, Any, Optional
from datetime import datetime
import json

# In-memory inventory database
inventory = {
    "LAPTOP001": {
        "name": "Dell Latitude 7420",
        "category": "Electronics",
        "quantity": 15,
        "min_threshold": 5,
        "price": 1200.00,
        "supplier": "Dell Technologies",
        "last_updated": "2025-01-15",
        "transactions": [
            {"date": "2025-01-10", "type": "purchase", "quantity": 20, "unit_cost": 1150.00},
            {"date": "2025-01-12", "type": "sale", "quantity": 5, "unit_price": 1200.00}
        ]
    },
    "CHAIR042": {
        "name": "Ergonomic Office Chair",
        "category": "Furniture",
        "quantity": 8,
        "min_threshold": 3,
        "price": 350.00,
        "supplier": "Office Depot",
        "last_updated": "2025-01-14",
        "transactions": [
            {"date": "2025-01-05", "type": "purchase", "quantity": 12, "unit_cost": 320.00},
            {"date": "2025-01-08", "type": "sale", "quantity": 4, "unit_price": 350.00}
        ]
    },
    "PAPER001": {
        "name": "A4 Copy Paper (500 sheets)",
        "category": "Office Supplies",
        "quantity": 2,
        "min_threshold": 10,
        "price": 8.99,
        "supplier": "Staples",
        "last_updated": "2025-01-13",
        "transactions": [
            {"date": "2025-01-01", "type": "purchase", "quantity": 50, "unit_cost": 7.50},
            {"date": "2025-01-10", "type": "sale", "quantity": 48, "unit_price": 8.99}
        ]
    },
    "MONITOR055": {
        "name": "Samsung 27\" 4K Monitor",
        "category": "Electronics",
        "quantity": 12,
        "min_threshold": 4,
        "price": 450.00,
        "supplier": "Samsung",
        "last_updated": "2025-01-16",
        "transactions": [
            {"date": "2025-01-15", "type": "purchase", "quantity": 15, "unit_cost": 400.00},
            {"date": "2025-01-16", "type": "sale", "quantity": 3, "unit_price": 450.00}
        ]
    }
}

# Create MCP server
mcp = FastMCP("InventoryManager")

# Tool: Check stock levels
@mcp.tool()
def check_stock(item_code: str) -> str:
    """Check current stock level and details for a specific item"""
    item = inventory.get(item_code)
    if not item:
        return f"Item code '{item_code}' not found in inventory."

    status_icon = "🔴" if item['quantity'] <= item['min_threshold'] else "🟡" if item['quantity'] <= item['min_threshold'] * 2 else "🟢"
    stock_status = "CRITICAL" if item['quantity'] <= item['min_threshold'] else "LOW" if item['quantity'] <= item['min_threshold'] * 2 else "GOOD"

    total_value = item['quantity'] * item['price']

    return f"""
{status_icon} Stock Report: {item['name']} ({item_code})
─────────────────────────────────────────────
Current Stock: {item['quantity']} units
Minimum Threshold: {item['min_threshold']} units
Status: {stock_status}
Unit Price: ${item['price']:.2f}
Total Value: ${total_value:.2f}
Supplier: {item['supplier']}
Last Updated: {item['last_updated']}
"""

# Tool: Add stock (purchase/restock)
@mcp.tool()
def add_stock(item_code: str, quantity: int, unit_cost: Optional[float] = None) -> str:
    """Add stock to inventory (purchase/restock operation)"""
    if item_code not in inventory:
        return f"Item code '{item_code}' not found. Use create_item() first."

    if quantity <= 0:
        return "Quantity must be greater than 0."

    item = inventory[item_code]
    old_quantity = item['quantity']
    item['quantity'] += quantity
    item['last_updated'] = datetime.now().strftime("%Y-%m-%d")

    # Record transaction
    cost = unit_cost or item['price'] * 0.85  # Default to 85% of selling price
    transaction = {
        "date": datetime.now().strftime("%Y-%m-%d"),
        "type": "purchase",
        "quantity": quantity,
        "unit_cost": cost
    }
    item['transactions'].append(transaction)

    return f"""
Stock Added Successfully!
─────────────────────────
Item: {item['name']} ({item_code})
Added: {quantity} units
Stock Level: {old_quantity} → {item['quantity']} units
Unit Cost: ${cost:.2f}
Total Cost: ${cost * quantity:.2f}
"""

# Tool: Remove stock (sale/usage)
@mcp.tool()
def remove_stock(item_code: str, quantity: int, unit_price: Optional[float] = None) -> str:
    """Remove stock from inventory (sale/usage operation)"""
    if item_code not in inventory:
        return f"Item code '{item_code}' not found."

    if quantity <= 0:
        return "Quantity must be greater than 0."

    item = inventory[item_code]
    if item['quantity'] < quantity:
        return f"Insufficient stock! Available: {item['quantity']}, Requested: {quantity}"

    old_quantity = item['quantity']
    item['quantity'] -= quantity
    item['last_updated'] = datetime.now().strftime("%Y-%m-%d")

    # Record transaction
    price = unit_price or item['price']
    transaction = {
        "date": datetime.now().strftime("%Y-%m-%d"),
        "type": "sale",
        "quantity": quantity,
        "unit_price": price
    }
    item['transactions'].append(transaction)

    # Check if stock is below threshold
    warning = ""
    if item['quantity'] <= item['min_threshold']:
        warning = f"\n WARNING: Stock below minimum threshold ({item['min_threshold']} units)!"

    return f"""
Stock Removed Successfully!
─────────────────────────
Item: {item['name']} ({item_code})
Removed: {quantity} units
Stock Level: {old_quantity} → {item['quantity']} units
Unit Price: ${price:.2f}
Total Revenue: ${price * quantity:.2f}{warning}
"""

# Tool: Get low stock alerts
@mcp.tool()
def get_low_stock_alerts() -> str:
    """Get list of items that are low on stock or below minimum threshold"""
    low_stock_items = []
    critical_items = []

    for code, item in inventory.items():
        if item['quantity'] <= item['min_threshold']:
            critical_items.append((code, item))
        elif item['quantity'] <= item['min_threshold'] * 2:
            low_stock_items.append((code, item))

    if not low_stock_items and not critical_items:
        return "All items are well-stocked!"

    alert_msg = "Stock Alerts\n" + "="*50 + "\n"

    if critical_items:
        alert_msg += "\nCRITICAL - Reorder Immediately:\n"
        for code, item in critical_items:
            alert_msg += f"  • {item['name']} ({code}): {item['quantity']} units (Min: {item['min_threshold']})\n"

    if low_stock_items:
        alert_msg += "\n🟡 LOW STOCK - Consider Reordering:\n"
        for code, item in low_stock_items:
            alert_msg += f"  • {item['name']} ({code}): {item['quantity']} units (Min: {item['min_threshold']})\n"

    return alert_msg

# Tool: Get inventory summary by category
@mcp.tool()
def get_inventory_summary(category: Optional[str] = None) -> str:
    """Get inventory summary, optionally filtered by category"""
    filtered_items = inventory.items()

    if category:
        filtered_items = [(code, item) for code, item in inventory.items()
                         if item['category'].lower() == category.lower()]
        if not filtered_items:
            return f"❌ No items found in category '{category}'"

    total_value = 0
    total_items = 0
    categories = {}

    summary = f"Inventory Summary"
    if category:
        summary += f" - {category.title()}"
    summary += "\n" + "="*60 + "\n"

    for code, item in filtered_items:
        item_value = item['quantity'] * item['price']
        total_value += item_value
        total_items += item['quantity']

        cat = item['category']
        if cat not in categories:
            categories[cat] = {'items': 0, 'value': 0, 'quantity': 0}
        categories[cat]['items'] += 1
        categories[cat]['value'] += item_value
        categories[cat]['quantity'] += item['quantity']

    # Category breakdown
    for cat, stats in categories.items():
        summary += f"\n{cat}:\n"
        summary += f"   Items: {stats['items']} types\n"
        summary += f"   Total Units: {stats['quantity']}\n"
        summary += f"   Total Value: ${stats['value']:,.2f}\n"

    summary += f"\n Grand Total: ${total_value:,.2f}"
    summary += f"\n Total Units: {total_items:,}"
    summary += f"\n Unique Items: {len(filtered_items)}"

    return summary

# Tool: Get transaction history
@mcp.tool()
def get_transaction_history(item_code: str, limit: int = 10) -> str:
    """Get recent transaction history for an item"""
    if item_code not in inventory:
        return f" Item code '{item_code}' not found."

    item = inventory[item_code]
    transactions = item['transactions']

    if not transactions:
        return f"No transaction history found for {item['name']} ({item_code})."

    # Sort by date (most recent first)
    sorted_transactions = sorted(transactions, key=lambda x: x['date'], reverse=True)
    limited_transactions = sorted_transactions[:limit]

    history = f"Transaction History: {item['name']} ({item_code})\n"
    history += "="*60 + "\n"

    for i, trans in enumerate(limited_transactions, 1):
        if trans['type'] == 'purchase':
            amount = f"${trans['unit_cost']:.2f} each"
            total = f"${trans['unit_cost'] * trans['quantity']:.2f}"
        else:
            amount = f"${trans['unit_price']:.2f} each"
            total = f"${trans['unit_price'] * trans['quantity']:.2f}"

        history += f"{i:2}. {trans['date']} | {trans['type'].title()} | "
        history += f"{trans['quantity']} units @ {amount} = {total}\n"

    if len(transactions) > limit:
        history += f"\n... and {len(transactions) - limit} more transactions"

    return history

# Tool: Create new item
@mcp.tool()
def create_item(item_code: str, name: str, category: str, initial_quantity: int,
                price: float, min_threshold: int, supplier: str) -> str:
    """Create a new item in the inventory"""
    if item_code in inventory:
        return f"Item code '{item_code}' already exists."

    if initial_quantity < 0 or price <= 0 or min_threshold < 0:
        return "Invalid values. Quantity and threshold must be non-negative, price must be positive."

    inventory[item_code] = {
        "name": name,
        "category": category,
        "quantity": initial_quantity,
        "min_threshold": min_threshold,
        "price": price,
        "supplier": supplier,
        "last_updated": datetime.now().strftime("%Y-%m-%d"),
        "transactions": []
    }

    # Add initial stock transaction if quantity > 0
    if initial_quantity > 0:
        inventory[item_code]['transactions'].append({
            "date": datetime.now().strftime("%Y-%m-%d"),
            "type": "purchase",
            "quantity": initial_quantity,
            "unit_cost": price * 0.8  # Assume 80% cost ratio
        })

    return f"""
New Item Created Successfully!
─────────────────────────────────
Item Code: {item_code}
Name: {name}
Category: {category}
Initial Stock: {initial_quantity} units
Price: ${price:.2f}
Min Threshold: {min_threshold} units
Supplier: {supplier}
"""

# Resource: Inventory dashboard
@mcp.resource("dashboard://inventory-overview")
def get_inventory_dashboard() -> str:
    """Get a comprehensive inventory dashboard"""
    total_items = len(inventory)
    total_value = sum(item['quantity'] * item['price'] for item in inventory.values())
    low_stock_count = sum(1 for item in inventory.values() if item['quantity'] <= item['min_threshold'])

    categories = {}
    for item in inventory.values():
        cat = item['category']
        if cat not in categories:
            categories[cat] = 0
        categories[cat] += 1

    dashboard = f"""
INVENTORY DASHBOARD
{'='*50}

Quick Stats:
  └─ Total Items: {total_items} types
  └─ Total Value: ${total_value:,.2f}
  └─ Low Stock Alerts: {low_stock_count} items

Categories:
"""

    for cat, count in categories.items():
        dashboard += f"  └─ {cat}: {count} items\n"

    dashboard += f"""
Action Required:
  └─ Items needing reorder: {low_stock_count}
  └─ Use get_low_stock_alerts() for details

Recent Activity:
  └─ Use get_transaction_history() for item details
"""

    return dashboard

# Resource: Supplier contact info
@mcp.resource("contacts://suppliers")
def get_supplier_contacts() -> str:
    """Get supplier contact information"""
    suppliers = set(item['supplier'] for item in inventory.values())

    # Mock contact info (in real app, this would be from a database)
    contact_info = {
        "Dell Technologies": "1-800-DELL-TECH | orders@dell.com",
        "Office Depot": "1-800-OFFICE | business@officedepot.com",
        "Staples": "1-800-STAPLES | supplies@staples.com",
        "Samsung": "1-800-SAMSUNG | b2b@samsung.com"
    }

    contacts = "Supplier Contact Directory\n" + "="*40 + "\n"

    for supplier in sorted(suppliers):
        contacts += f"\n{supplier}\n"
        contacts += f"   {contact_info.get(supplier, ' Contact info not available')}\n"

    return contacts

if __name__ == "__main__":
    mcp.run()

Writing server.py


# Creating Client-Side Logic using Langgraph react agent

## Configuring the LLM

In [14]:
# Load the JSON file and extract values
file_name = 'config.json'                                                       # Name of the configuration file
with open(file_name, 'r') as file:                                              # Open the config file in read mode
    config = json.load(file)                                                    # Load the JSON content as a dictionary
    OPENAI_API_KEY = config.get("OPENAI_API_KEY")                                             # Extract the API key from the config
    OPENAI_API_BASE = config.get("OPENAI_API_BASE")                             # Extract the OpenAI base URL from the config

In [15]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_API_BASE,
    model='gpt-4o-mini',
    temperature=0
)

The code sets up standard input/output server parameters to execute the `server.py` script using Python, enabling integration and interaction with the inventory management server.

In [16]:
server_params = StdioServerParameters(
    command="python",
    args=['server.py']
)

 This function is used to pull a prompt template named **react** from the LangChain Hub.

In [17]:
prompt = hub.pull("hwchase17/react")

The `main` function is an asynchronous routine that handles a query (`user_query`) by communicating with a server using standard input/output channels, facilitated by a subprocess. It establishes an MCP (Multi-Channel Processing) session to discover and load tools that the server offers. Within this session, a ReAct agent is created, utilizing a large language model (LLM) and loaded tools, and it processes the input query to generate a response. The function ensures error handling both within the client session context and during the setup/teardown phases of the stdio client, outputting errors and tracing as necessary.

A ReAct agent is an AI system that combines reasoning and acting by iteratively generating thoughts, performing actions using tools, and observing the results to achieve complex goals.

In [18]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

async def main(user_query):
    print(f"Starting MCP client for query: '{user_query}'")
    agent_response = None
    try:
        async with stdio_client(server_params, errlog=subprocess.PIPE) as (read, write):
            print("stdio_client connected. Initializing ClientSession...")
            try:
                async with ClientSession(read, write) as session:
                    await session.initialize()
                    print("ClientSession initialized. Loading tools...")
                    tools = await load_mcp_tools(session)
                    print(f"Tools loaded: {[tool.name for tool in tools]}")
                    print("Creating LangGraph agent...")

                    # Use LangGraph's create_react_agent instead of LangChain's
                    agent = create_react_agent(llm, tools)

                    print("Invoking agent...")
                    # Use proper message format for LangGraph
                    agent_response = await agent.ainvoke({
                        "messages": [HumanMessage(content=user_query)]
                    })
                    print("Agent finished.")
            except Exception as session_err:
                print(f"\n--- ERROR within ClientSession context ---")
                import traceback
                traceback.print_exc()
                agent_response = {"error": "ClientSession failed", "details": str(session_err)}

        return agent_response
    except Exception as e:
        print(f"\n--- ERROR: An unexpected error occurred outside ClientSession ---")
        import traceback
        traceback.print_exc()
        print("---------------------------------------------------------------")
        return {"error": "Agent execution failed unexpectedly", "details": str(e)}


# Testing the Agent

In [19]:
response = asyncio.run(main("What's the current status of our inventory?"))

Starting MCP client for query: 'What's the current status of our inventory?'
stdio_client connected. Initializing ClientSession...
ClientSession initialized. Loading tools...
Tools loaded: ['check_stock', 'add_stock', 'remove_stock', 'get_low_stock_alerts', 'get_inventory_summary', 'get_transaction_history', 'create_item']
Creating LangGraph agent...
Invoking agent...


/tmp/ipython-input-3696069522.py:19: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


Agent finished.


In [20]:
print(response['messages'][-1].content)

Here's the current status of our inventory:

### Inventory Summary
- **Electronics:**
  - Items: 2 types
  - Total Units: 27
  - Total Value: $23,400.00

- **Furniture:**
  - Items: 1 type
  - Total Units: 8
  - Total Value: $2,800.00

- **Office Supplies:**
  - Items: 1 type
  - Total Units: 2
  - Total Value: $17.98

### Grand Total:
- Total Value: $26,217.98
- Total Units: 37
- Unique Items: 4


In [21]:
response = asyncio.run(main("Check the stock level for item PAPER001"))

Starting MCP client for query: 'Check the stock level for item PAPER001'
stdio_client connected. Initializing ClientSession...
ClientSession initialized. Loading tools...
Tools loaded: ['check_stock', 'add_stock', 'remove_stock', 'get_low_stock_alerts', 'get_inventory_summary', 'get_transaction_history', 'create_item']
Creating LangGraph agent...
Invoking agent...


/tmp/ipython-input-3696069522.py:19: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


Agent finished.


In [22]:
print(response['messages'][-1].content)

The stock level for **A4 Copy Paper (500 sheets)** (Item Code: PAPER001) is as follows:

- **Current Stock:** 2 units
- **Minimum Threshold:** 10 units
- **Status:** CRITICAL
- **Unit Price:** $8.99
- **Total Value:** $17.98
- **Supplier:** Staples
- **Last Updated:** January 13, 2025

It is recommended to restock this item as it is below the minimum threshold.


In [23]:
response = asyncio.run(main("Are there any critical stock alerts I should know about?"))

Starting MCP client for query: 'Are there any critical stock alerts I should know about?'
stdio_client connected. Initializing ClientSession...
ClientSession initialized. Loading tools...
Tools loaded: ['check_stock', 'add_stock', 'remove_stock', 'get_low_stock_alerts', 'get_inventory_summary', 'get_transaction_history', 'create_item']
Creating LangGraph agent...
Invoking agent...


/tmp/ipython-input-3696069522.py:19: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


Agent finished.


In [24]:

print(response['messages'][-1].content)

There is a critical stock alert that you should be aware of:

- **A4 Copy Paper (500 sheets) (PAPER001)**: Only 2 units left (Minimum threshold: 10). 

You should reorder this item immediately.


In [25]:
response = asyncio.run(main("Give me a summary of all furniture items"))

Starting MCP client for query: 'Give me a summary of all furniture items'
stdio_client connected. Initializing ClientSession...
ClientSession initialized. Loading tools...
Tools loaded: ['check_stock', 'add_stock', 'remove_stock', 'get_low_stock_alerts', 'get_inventory_summary', 'get_transaction_history', 'create_item']
Creating LangGraph agent...
Invoking agent...


/tmp/ipython-input-3696069522.py:19: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


Agent finished.


In [26]:
print(response['messages'][-1].content)

### Inventory Summary - Furniture

- **Total Types of Items:** 1
- **Total Units:** 8
- **Total Value:** $2,800.00

**Grand Total:**
- **Total Value:** $2,800.00
- **Total Units:** 8
- **Unique Items:** 1


In [27]:
response = asyncio.run(main("What's the total value of our current inventory?"))

Starting MCP client for query: 'What's the total value of our current inventory?'
stdio_client connected. Initializing ClientSession...
ClientSession initialized. Loading tools...
Tools loaded: ['check_stock', 'add_stock', 'remove_stock', 'get_low_stock_alerts', 'get_inventory_summary', 'get_transaction_history', 'create_item']
Creating LangGraph agent...
Invoking agent...


/tmp/ipython-input-3696069522.py:19: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


Agent finished.


In [28]:
print(response['messages'][-1].content)

The total value of the current inventory is **$26,217.98**.


# Conclusion

Observe how the introduction of MCP has significantly simplified client-side operations. Previously, each function intended for use by agents needed to be configured individually on the client side, which added complexity and hindered usability. With MCP, clients can now seamlessly integrate with the MCP server, streamlining the process and enhancing the efficiency of incorporating these functions into their agentic systems.